In [108]:
from pde_rk import pde_rk
import numpy as np
import pandas as pd
import plotly.express as px

## PDE model of advection, diffusion and membrane exchange

Aiming to investigate the efffects of membrane diffusion and membrane-cytoplasm exchange on segregation efficiency. This notebook shows code for running single simulations with a specified D and koff.

### Cortical flow profile

We will use the following cortical flow profile in a system 60 µm in length (anterior pole to posterior pole), which is based on measurements of nmy-2 velocity. Positive values reflect movement from the posterior to the anterior.

In [109]:
def generate_flow_profile(A=74, B=391, C=1000, D=100):
    X = np.linspace(0, 60, 100)
    v = (((60-X) / A) * np.exp(-((60-X) ** 2) / B)) - ((X / C) * np.exp(-(X ** 2) / D))
    return v

In [110]:
fig = px.line(x=np.linspace(0, 60, 100), y=generate_flow_profile(), 
              labels={"x": "Position (x)", "y": "Flow velocity (µm/s)"}, width=500, height=300)
fig.show()


### Building a PDE model

To build and solve a PDE model we will use the pde_rk function found in the the _pre_rk_ package. This is a versatile PDE solver that can be used to solve any 1D system of PDEs, using an adaptive Runge-Kutta method.

In [111]:
# Uncomment to see documentation:
# help(pde_rk) 

The full PDE model is built as a Python class:

In [112]:
class Model:
    def __init__(self, D, kon, koff, psi=0.174, tot=1.56):

        # Diffusion
        self.D = D # diffusion coefficient

        # Flow profile
        self.flow_profile = generate_flow_profile()

        # Membrane exchange
        self.kon = kon # membrane binding rate
        self.koff = koff # membrane unbinding rate

        # Misc
        self.tot = tot # total amount of protein
        self.psi = psi # surface area to volume ratio
        self.xsteps = 100 # number of positions to split the spatial dimension into
        self.L = 60 # system length
        self.deltax = self.L / self.xsteps # spatial step

    def diffusion(self, concs, dx):
        concs_ = np.r_[concs[0], concs, concs[-1]] # Dirichlet boundary conditions
        d = concs_[:-2] - 2 * concs_[1:-1] + concs_[2:]
        return d / (dx ** 2)

    def flow(self, concs, dx):
        # Calculates the gradient in both directions and takes the average
        return (np.r_[0, np.diff(concs * self.flow_profile)] +
                  np.r_[np.diff(concs * self.flow_profile), 0]) / (2 * dx)

    def dxdt(self, X):
        m = X[0]
        c = self.tot - self.psi * np.mean(m) # calculate uniform cytoplasmic concentration
        flow = self.flow(m, self.deltax)
        dm = (self.kon * c) - (self.koff * m) + (self.D * self.diffusion(m, self.deltax)) + flow
        return [dm, ]

    def dxdt_no_flow(self, X):
        m = X[0]
        c = self.tot - self.psi * np.mean(m) # calculate uniform cytoplasmic concentration
        dm = (self.kon * c) - (self.koff * m) + (self.D * self.diffusion(m, self.deltax))
        return [dm, ]

    def run(self, Tmax, t_eval=None, start=None, flow=True, killfunc=None, maxstep=None, rk=True):

        # Specify evaluation times
        if t_eval is None:
            t_eval = np.arange(0, Tmax + 0.0001, Tmax)

        # Specify starting conditions
        if start is None:
            # Start from uniform equilibrium membrane concentration, calculated analytically
            start = (self.kon * self.tot) / (self.koff + self.psi * self.kon)
            start *= np.ones([self.xsteps])

        # Specify flow regime
        if flow:
            func = self.dxdt
        else:
            func = self.dxdt_no_flow

        # Run simulation
        soln, time, solns, times = pde_rk(dxdt=func, X0=[start, ], Tmax=Tmax, deltat=0.01, t_eval=t_eval, 
                                         killfunc=killfunc, maxstep=maxstep)
        # Return results
        return soln, time, solns, times

### Simulate segregation

Starting from a uniform equilibrium state, systems are simulated with the cortical flow profile shown above. Specify the diffusion coefficient and off rate below

In [113]:
# Specify parameters
D = 0.1 ### <- diffusion coefficient on the membrane (µm^2/s)
koff = 0.005 ### <- membrane unbinding rate (/s)

# Build the class
m = Model(D=D, koff=koff, kon=koff)

# Specify simulation length (in seconds)
Tmax_seg = 500 

# Run simulation (flow=True)
soln_seg, time_seg, solns_seg, times_seg = m.run(Tmax=Tmax_seg, flow=True,
                                                 t_eval=np.arange(0, Tmax_seg + 1, 10))

# Create dataframe
df = pd.DataFrame({"Time (s)": times_seg.astype(int), "Membrane Concentration": list(solns_seg[0]), 
                   "Position": [np.linspace(0, 60, 100)] * len(times_seg)}).explode(column=["Membrane Concentration", "Position"])

In [114]:
# Interactive figure of simulation results
fig = px.line(data_frame=df, x="Position", y='Membrane Concentration', animation_frame='Time (s)', range_y=[0, 1.1 * np.max(solns_seg[0])],
              width=500, height=400)
fig["layout"].pop("updatemenus")
fig.show()

#### Score final asymmetry index (ASI)

In [115]:
def calc_asi(m):
    ant = np.mean(m[:30])
    post = np.mean(m[-30:])
    asi = abs((ant - post) / (2 * (ant + post)))
    return asi

asi_seg = calc_asi(soln_seg[0])
print('ASI = %.3f' % asi_seg)

ASI = 0.254


### Simulate relaxation

Starting from the final state of the above simulation, flows are switched off and the system is left to relax

In [116]:
# Specify simulation length (in seconds)
Tmax_rel = 1000 

# Run simulation (flow=False)
soln_rel, time_rel, solns_rel, times_rel = m.run(start=soln_seg[0], flow=False, Tmax=Tmax_rel,
                                                t_eval=np.arange(0, Tmax_rel + 1, 10))  

# Create dataframe
df2 = pd.DataFrame({"Time (s)": times_rel.astype(int), "Membrane Concentration": list(solns_rel[0]), 
                   "Position": [np.linspace(0, 60, 100)] * len(times_rel)}).explode(column=["Membrane Concentration", "Position"])  

In [117]:
# Interactive figure of simulation results
fig = px.line(data_frame=df2, x="Position", y='Membrane Concentration', animation_frame='Time (s)', range_y=[0, 1.1 * np.max(solns_rel[0])],
              width=500, height=400)
fig["layout"].pop("updatemenus")
fig.show()

### Calculate timescale of relaxation

Calculating the time for ASI to decrease to half of its initial value. To do so we will use the killfunc option in the pde_rk function to simulate the system until ASI has decreased by half. We will also set maxstep=0.1 to limit the maximum time step so that times are calculated to the nearest 0.1 second.

In [118]:
# Set up kill function
def killfunc(X, asi_thresh=asi_seg * 0.5):
    current_asi = calc_asi(X[0])
    if current_asi < asi_thresh:
        return True
    return False

# Run simulation (flow=False)
_, relaxation_time, _, _  = m.run(Tmax=10000, start=soln_seg[0], flow=False, killfunc=killfunc, maxstep=0.1)

# Print relaxation time (seconds)
print('Relaxation time = %.1f seconds' % relaxation_time)

Relaxation time = 108.7 seconds


### Two species model

For the model with a mix of fast and slow exchanging states (figure S4), see [this notebook](notebook_two_species.ipynb)